In [1]:
# arrival_spotter
# purpose: takes a list of arrival predictions from GetStopPredictions.jsp (live or from db) 
# and returns a list of arrivals for all time along an entire route

# usage: use with stopwatcher to TK...

In [2]:
# algorithm
#
# 1. slice all observations into by vehicle lists
# 2. sort by day, trip/run, stop --> "an approach"
# 3. for each approach, sort observations by time and keep latest 'approaching' as the observed arrival time
# 4. dump a table of all the arrival times for that vehicle along its trip

In [3]:
import src.buses.Buses as Buses
from src.buses.reportcard_helpers import *

In [4]:
import time
import pandas as pd

In [5]:
source = 'nj'
route = 119
stop = 30189

In [6]:
(conn, db) = db_setup(route)

In [7]:
# only get buses APPROACHING A STOP
arrival_query = ('SELECT * FROM stop_predictions \
              WHERE (rd = %s AND stop_id = %s AND pt = "APPROACHING") \
              ORDER BY timestamp;' % (route, stop))
# arrival_query = ('SELECT * FROM stop_predictions;')

In [8]:
df = pd.read_sql_query(arrival_query, conn)

In [14]:
timestamp_fix(df)
df.head()

,pkey,cars,consist,fd,m,name,pt,rd,rn,scheduled,stop_id,stop_name,v,timestamp
0,8774,,,119 NEW YORK,1,StopPrediction,APPROACHING,119,119,false,30189,CONGRESS ST + WEBSTER AVE,6201,2018-06-07 15:05:10
1,21853,,,119 NEW YORK,1,StopPrediction,APPROACHING,119,119,false,30189,CONGRESS ST + WEBSTER AVE,6017,2018-06-07 16:29:07
2,28533,,,119 NEW YORK,1,StopPrediction,APPROACHING,119,119,false,30189,CONGRESS ST + WEBSTER AVE,6029,2018-06-07 17:02:06
3,34702,,,119 NEW YORK,1,StopPrediction,APPROACHING,119,119,false,30189,CONGRESS ST + WEBSTER AVE,6015,2018-06-07 17:29:11
4,43210,,,119 NEW YORK,1,StopPrediction,APPROACHING,119,119,false,30189,CONGRESS ST + WEBSTER AVE,6022,2018-06-07 18:04:10


In [15]:
# 97 observations to work with
df.shape

(97, 14)

In [22]:
# drop a ton of columns
df = df.drop(columns=['cars','consist','fd','m','name','rn','scheduled','stop_name'])
df.head()

,pkey,pt,rd,stop_id,v,timestamp
0,8774,APPROACHING,119,30189,6201,2018-06-07 15:05:10
1,21853,APPROACHING,119,30189,6017,2018-06-07 16:29:07
2,28533,APPROACHING,119,30189,6029,2018-06-07 17:02:06
3,34702,APPROACHING,119,30189,6015,2018-06-07 17:29:11
4,43210,APPROACHING,119,30189,6022,2018-06-07 18:04:10


In [ ]:
# find the last 'APPROACHING' observation in each 'approach' sequence

In [ ]:
# build a report card for just this one stop, plotting arrivals on a week timeline, and some basic statistics


### more comprehensive --- new data structure, building up list of visits per bus per stop all history

In [10]:
# THIS IS WAYYYYYYY FASTER !!!

# dicts of when a specific vehicle has called at a specific stop over all time

# history data structure = assign every row in df to a dict keyed to unique vehicle/stop_id instance
# {
#  5403_20640: [row1, row15, ...]
#  5704_20933: [row2, row37, ...]
#  unique_bus_stop_combo: [visit, visit, visit, ...]
# }

from collections import defaultdict
history = defaultdict(list)

for index,row in df.iterrows():
    key = row['v'] + '_' + row['stop_id']
    history[key].append(row)


In [11]:
history['5241_20581']

[]

In [12]:
# turn each into a list or df

5241_visits_20581=[]
for visit in history['5241_20581']:
    5241_visits_20581.append(timestamp)
    # compare each timestamp to next
    # if next is more than 5 minutes ahead, keep this one -- e.g. this is last in a sequence
    


SyntaxError: invalid syntax (<ipython-input-12-4ece37be126e>, line 3)

In [ ]:
# make list of vehicles in use
vehicles=[]
for index,row in df.iterrows():
    vehicles.append(row['v'])
    
# get list of stops in route
stoplist = get_stoplist(route)

# dump the visit log
for stop_id in stoplist:
    print "stop %s " % stop_id
    
    # cull list of vehicles
    
    for vehicle in vehicles:
        key = vehicle + '_' + stop_id
        print "\t bus %s" % vehicle
        vehicle_visits = history[key]
        
        for visit in vehicle_visits:
            print "\t\t visit %s" % visit.timestamp
    print
        

### to do

In [ ]:
# 1. change vehicle list -- 
#
# 2.
#
# 3. 
#

# if gap if more than 5 minutes then its a different run gap


### start here ---------------------------------------------------

In [ ]:
# # old stopwatcher
# # should run as a cron job
# # python stopwatcher.py -s nj -r 87

# # all this program does is fetch
# # the current arrivals for every stop on a source, route
# # and stick it in the database

# import buses.Buses as Buses
# from buses.reportcard_helpers import *

# import argparse, sys, datetime


# def fetch_arrivals(source, route):

#     (conn, db) = db_setup(route)

#     routedata = Buses.parse_route_xml(Buses.get_xml_data(source, 'routes', route=route))

#     stoplist = []

#     for rt in routedata:
#         for path in rt.paths:
#             for p in path.points:
#                 if p.__class__.__name__ == 'Stop':
#                     stoplist.append(p.identity)

#     for s in stoplist:
#         arrivals = Buses.parse_stopprediction_xml(Buses.get_xml_data('nj', 'stop_predictions', stop=s, route=route))
#         # sys.stdout.write('.')
#         now = datetime.datetime.now()
#         db.insert_positions(arrivals, now)

#     return


# def main():

#     parser = argparse.ArgumentParser()
#     parser.add_argument('-s', '--source', dest='source', required=True, default='nj', help='source name')
#     parser.add_argument('-r', '--route', dest='route', required=True, help='route # ')
#     args = parser.parse_args()

#     fetch_arrivals(args.source, args.route)

# if __name__ == "__main__":
#     main()
